In [173]:
import yaml
import pandas as pd
import re

## 2. Data acquisition

The data used in this project is obtained from two different data sources. The BMI data is obtained from https://www.ncdrisc.org/data-downloads-adiposity.html and downloaded in the csv format. This data contains out of all of the mean BMI's (with 95% confidence intervals) per country per sex, ranging from the year 1975 to 2016. In addition, the dataset contains the prevalence of certain BMI ranges (with 95% confidence intervals). For example, a person is classified as obese if the BMI is higher or equal to 30.

The affordability of a healthy diet is obtained from https://databank.worldbank.org/source/food-prices-for-nutrition (select all under country, select 'Affordability of a healthy diet: ratio of cost to food expenditures' under series, select 2017 under year) and downloaded in the csv format. In this dataset the affordability of a healthy diet is defined as the ratio of the cost of a healthy diet to food expenditures. This ratio is given by country and by sex in the year 2017.

Keep in mind that the BMI dataset is from the year 2017 and the healthy diet affordability dataset is from 2016. At this point in time the global BMI data is only available up to 2016 and the healthy diet data from 2017 until 2020. In this project the 2017 and 2016 will be compared. The mean BMI and heatlhy diet affordability are unlikely to change a lot from year to year and therefor are compared. However, when 2017 BMI data is available it is better to use that instead of the 2016 data.

## 3. Loading the data

The dataset are loaded into a pands dataframe using a config file

In [174]:
# load the datafiles using a config files
with open("./config.yaml", 'r') as stream:
    config = yaml.safe_load(stream)

bmi = config['bmi']
healthy_diet_affordability = config['food_affordability']

### 3.1 BMI

In [175]:
df_bmi = pd.read_csv(bmi, encoding='latin-1')  # file uses latin-1 encoding
df_bmi = df_bmi[df_bmi.Year == 2016]  # we only want the 2016 data
df_bmi = df_bmi.drop(columns='Year')  # remove the year column, we only have 2016 left so redundant
df_bmi.head()

,Country/Region/World,ISO,Sex,Mean BMI,Mean BMI lower 95% uncertainty interval,Mean BMI upper 95% uncertainty interval,Prevalence of BMI>=30 kg/m² (obesity),Prevalence of BMI>=30 kg/m² lower 95% uncertainty interval,Prevalence of BMI>=30 kg/m² upper 95% uncertainty interval,Prevalence of BMI>=35 kg/m² (severe obesity),...,Prevalence of BMI 25 kg/m² to <30 kg/m² upper 95% uncertainty interval,Prevalence of BMI 30 kg/m² to <35 kg/m²,Prevalence of BMI 30 kg/m² to <35 kg/m² lower 95% uncertainty interval,Prevalence of BMI 30 kg/m² to <35 kg/m² upper 95% uncertainty interval,Prevalence of BMI 35 kg/m² to <40 kg/m²,Prevalence of BMI 35 kg/m² to <40 kg/m² lower 95% uncertainty interval,Prevalence of BMI 35 kg/m² to <40 kg/m² upper 95% uncertainty interval,Prevalence of BMI >=40 kg/m²(morbid obesity),Prevalence of BMI >=40 kg/m² lower 95% uncertainty interval,Prevalence of BMI >=40 kg/m² upper 95% uncertainty interval
41,Afghanistan,AFG,Men,22.682456,20.157475,25.241857,0.033603,0.013884,0.066334,0.003314,...,0.242503,0.030290,0.011207,0.062681,0.002271,0.000310,0.007487,0.001043,0.000074,0.004265
83,Albania,ALB,Men,27.174471,25.975170,28.338256,0.223735,0.153334,0.300834,0.045036,...,0.515957,0.178699,0.113401,0.252200,0.037684,0.013616,0.076984,0.007352,0.001180,0.021953
125,Algeria,DZA,Men,24.865386,23.487321,26.220294,0.206662,0.141854,0.279979,0.042840,...,0.463570,0.163822,0.102610,0.234048,0.031750,0.011066,0.065164,0.011090,0.002289,0.029802
167,American Samoa,ASM,Men,33.066721,31.338678,34.662447,0.587546,0.502606,0.666355,0.322678,...,0.370043,0.264868,0.191211,0.340031,0.183871,0.109862,0.263723,0.138807,0.067651,0.223666
209,Andorra,AND,Men,27.478395,24.988831,30.001977,0.267498,0.186223,0.354723,0.068565,...,0.532478,0.198934,0.125035,0.280761,0.052701,0.017627,0.109639,0.015864,0.002652,0.046309


Some column names are pretty long and make it inconvenient to read and work with. The next code chunk shortens these column names to make it more readible. This is done by regex and substituting some words into shorter words or removing parts of the column name.

In [176]:
# dict for patterns and replacements for the column names
replace_dict = {'Prevalence': 'Prev', 'kg/m²': '', 'lower 95% uncertainty interval': 'lower', 'upper 95% uncertainty interval': 'upper'}
for pattern, replacement in replace_dict.items():
    df_bmi = df_bmi.rename(columns=lambda column: re.sub(pattern, replacement, column))
    

In [177]:
df_bmi.head()

,Country/Region/World,ISO,Sex,Mean BMI,Mean BMI lower,Mean BMI upper,Prev of BMI>=30 (obesity),Prev of BMI>=30 lower,Prev of BMI>=30 upper,Prev of BMI>=35 (severe obesity),...,Prev of BMI 25 to <30 upper,Prev of BMI 30 to <35,Prev of BMI 30 to <35 lower,Prev of BMI 30 to <35 upper,Prev of BMI 35 to <40,Prev of BMI 35 to <40 lower,Prev of BMI 35 to <40 upper,Prev of BMI >=40 (morbid obesity),Prev of BMI >=40 lower,Prev of BMI >=40 upper
41,Afghanistan,AFG,Men,22.682456,20.157475,25.241857,0.033603,0.013884,0.066334,0.003314,...,0.242503,0.030290,0.011207,0.062681,0.002271,0.000310,0.007487,0.001043,0.000074,0.004265
83,Albania,ALB,Men,27.174471,25.975170,28.338256,0.223735,0.153334,0.300834,0.045036,...,0.515957,0.178699,0.113401,0.252200,0.037684,0.013616,0.076984,0.007352,0.001180,0.021953
125,Algeria,DZA,Men,24.865386,23.487321,26.220294,0.206662,0.141854,0.279979,0.042840,...,0.463570,0.163822,0.102610,0.234048,0.031750,0.011066,0.065164,0.011090,0.002289,0.029802
167,American Samoa,ASM,Men,33.066721,31.338678,34.662447,0.587546,0.502606,0.666355,0.322678,...,0.370043,0.264868,0.191211,0.340031,0.183871,0.109862,0.263723,0.138807,0.067651,0.223666
209,Andorra,AND,Men,27.478395,24.988831,30.001977,0.267498,0.186223,0.354723,0.068565,...,0.532478,0.198934,0.125035,0.280761,0.052701,0.017627,0.109639,0.015864,0.002652,0.046309


### 3.2 Affordability of a healthy diet

In [178]:
df_healthy_diet = pd.read_csv(healthy_diet_affordability, encoding='latin-1', skipfooter=5, engine='python')  # skip last lines, does not contain data
df_healthy_diet.drop(columns=['Classification Name', 'Classification Code', 'Time', 'Time Code'], inplace=True)  # irrelevant columns
df_healthy_diet.rename(columns={'Affordability of a healthy diet: ratio of cost to food expenditures [CoHD_fexp]': 'Affordability of a healthy diet'}, inplace=True)

In [196]:
df_healthy_diet.head()

,Country Name,Country Code,Affordability of a healthy diet
0,Albania,ALB,0.425
1,Algeria,DZA,0.605
2,Angola,AGO,0.972
3,Anguilla,AIA,0.577
4,Antigua and Barbuda,ATG,0.767


## 4. Data exploration

In [193]:
countries_bmi = df_bmi['ISO'].unique()
countries_healthy_diet = df_healthy_diet['Country Code'].unique()
countries_not_in_common = set(countries_bmi) ^ set(countries_healthy_diet)
print(countries_not_in_common)
print(len(countries_not_in_common))

{'NIU', 'PLW', 'HIC', 'AFG', 'CUW', 'KIR', 'BON', 'GEO', 'AIA', 'AND', 'MEA', 'TCA', 'ECS', 'LBN', 'VUT', 'UMC', 'TKL', 'MSR', 'TON', 'LIC', 'EAS', 'COK', 'TLS', 'WSM', 'SYR', 'SAS', 'SOM', 'TUV', 'PRI', 'YEM', 'UZB', 'FSM', 'PNG', 'ABW', 'VEN', 'LCN', 'CUB', 'LBY', 'CYM', 'PYF', 'WLD', 'ERI', 'GTM', 'LMC', 'GRL', 'SLB', 'MHL', 'TKM', 'SXM', 'ASM', 'SSF', 'UKR', 'PRK', 'NRU', 'VGB', 'NAC'}
56


As can be seen above there are a lot of countries (56) which are not in common between both datasets. When merging these datasets (using inner merge) this data gets lost. Therefore it makes sense to merge the datasets before data exploration, because otherwise data exploration is done on data which is later omitted.

In [195]:
df_merged = df_healthy_diet.merge(right=df_bmi, 
                                  left_on='Country Code',
                                  right_on='ISO',
                                  how='inner')
df_merged.drop(columns=['Country/Region/World', 'ISO'])  # duplicate columns
df_merged.head()

,Country Name,Country Code,Affordability of a healthy diet,Country/Region/World,ISO,Sex,Mean BMI,Mean BMI lower,Mean BMI upper,Prev of BMI>=30 (obesity),...,Prev of BMI 25 to <30 upper,Prev of BMI 30 to <35,Prev of BMI 30 to <35 lower,Prev of BMI 30 to <35 upper,Prev of BMI 35 to <40,Prev of BMI 35 to <40 lower,Prev of BMI 35 to <40 upper,Prev of BMI >=40 (morbid obesity),Prev of BMI >=40 lower,Prev of BMI >=40 upper
0,Albania,ALB,0.425,Albania,ALB,Men,27.174471,25.975170,28.338256,0.223735,...,0.515957,0.178699,0.113401,0.252200,0.037684,0.013616,0.076984,0.007352,0.001180,0.021953
1,Albania,ALB,0.425,Albania,ALB,Women,26.507512,25.196840,27.859854,0.227215,...,0.365555,0.154878,0.096033,0.225306,0.054146,0.022398,0.100402,0.018191,0.004656,0.044692
2,Algeria,DZA,0.605,Algeria,DZA,Men,24.865386,23.487321,26.220294,0.206662,...,0.463570,0.163822,0.102610,0.234048,0.031750,0.011066,0.065164,0.011090,0.002289,0.029802
3,Algeria,DZA,0.605,Algeria,DZA,Women,26.561166,25.080506,28.031641,0.362187,...,0.383587,0.225826,0.155871,0.303311,0.093082,0.047469,0.152826,0.043279,0.015501,0.089878
4,Angola,AGO,0.972,Angola,AGO,Men,22.436538,19.732903,25.172488,0.042276,...,0.237444,0.035127,0.011140,0.073699,0.005868,0.000739,0.018757,0.001281,0.000033,0.006267


### 4.1 BMI

In [180]:
df_bmi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 41 to 16799
Data columns (total 33 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Country/Region/World               400 non-null    object 
 1   ISO                                400 non-null    object 
 2   Sex                                400 non-null    object 
 3   Mean BMI                           400 non-null    float64
 4   Mean BMI lower                     400 non-null    float64
 5   Mean BMI upper                     400 non-null    float64
 6   Prev of BMI>=30  (obesity)         400 non-null    float64
 7   Prev of BMI>=30  lower             400 non-null    float64
 8   Prev of BMI>=30  upper             400 non-null    float64
 9   Prev of BMI>=35  (severe obesity)  400 non-null    float64
 10  Prev of BMI>=35  lower             400 non-null    float64
 11  Prev of BMI>=35  upper             400 non-null    floa

As can be seen there are 400 entries in total, since every country contains data about both sexes the dataset contains 200 countries. Fortunately, the dataset contains no missing values.

In [181]:
print(df_bmi['Country/Region/World'].nunique())  # confirming that there are indeed 200 countries

200


In [182]:
df_bmi['Mean BMI'].describe()

count    400.000000
mean      26.122793
std        2.674200
min       20.110440
25%       24.153777
50%       26.349274
75%       27.636261
max       35.148865
Name: Mean BMI, dtype: float64

### Affordability of a healthy diet